In [ ]:
from rapidfuzz import process as rf_process
from rapidfuzz import fuzz as rf
import pandas as pd


df1 = pd.read_csv('./data/전국연도별방문자추이.csv', encoding='euc-kr')
df2 = pd.read_csv('./data/회계정보전처리.csv', encoding='utf-8')

index_col1 = 'index'  
index_col2 = 'index'  

#들어간 값을 정규화. 공백 삭제, nan 값이면 빈칸 반환. _로 구분되어있으면 "_", ""를 사용
def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).lower().replace(" ", "")

df1['normalized_index'] = df1[index_col1].apply(normalize_text)
df2['normalized_index'] = df2[index_col2].apply(normalize_text)


# rapidfuzz에서 유사도를 측정한 socrer를 기준으로 추가적인 dataset 생성. 비교할 데이터셋 두개의 정보를 모두 담음.
similarities = []
for idx1, original_index in zip(df1['normalized_index'], df1[index_col1]):
    match = rf_process.extractOne(idx1, df2['normalized_index'], scorer=rf.ratio)
    similarities.append({
        'df1_original': original_index,
        'df1_normalized': idx1,
        'df2_best_match': match[0],
        'df2_original': df2[df2['normalized_index'] == match[0]][index_col2].values[0],
        'similarity_score': match[1]
    })

similarities_df = pd.DataFrame(similarities)

# 유사도 기준 필터링 (예: 80 이상)
# 위의 데이터는 하나라도 같다면 모두 담기 때문에, 아래의 스코어를 통해서 일정 이상 유사도를 지닌 것들만 남김.
filtered_df = similarities_df[similarities_df['similarity_score'] >= 80]

filtered_df.to_csv('./data/matched_similarities.csv', index=False, encoding='utf-8-sig')


In [12]:
matched_df = pd.read_csv('./data/matched_similarities.csv')

df2_matched = pd.merge(
    df1, 
    matched_df[['df1_original', 'df2_original']], 
    left_on='index',  
    right_on='df1_original',  
    how='left'  
)


final_df = pd.merge(
    df2_matched,  
    df2,  
    left_on='df2_original',  
    right_on='index', 
    how='left' 
)

print(final_df.head())

final_df.to_csv('./data/전국연도별방문자회계정보.csv', index=False, encoding='utf-8-sig')

      축제명  개최년도  축체기간(일)  (현지인)방문자수  (외지인)방문자수  (외국인)방문자수  (전체)방문자수  일평균 방문자수  \
0  강경젓갈축제  2018        5    50642.0    59291.0      108.0  110041.0  22008.20   
1  강경젓갈축제  2022        5    57971.0    69910.0       19.0  127900.0  25580.00   
2  강경젓갈축제  2023        4    52282.0    71999.0       18.0  124299.0  31074.75   
3  강릉커피축제  2018        5    61368.0    82621.0      279.0  144268.0  28853.60   
4  강릉커피축제  2019        4    98907.0    61246.0      743.0  160896.0  40224.00   

   전년도 일평균 방문자수  일평균 방문자수 증감률  ...    회계연도  지역명   행사축제명           총비용  \
0           NaN           NaN  ...  2018.0   충남  강경젓갈축제  8.250000e+08   
1       22008.2          16.2  ...  2022.0   충남  강경젓갈축제  1.003000e+09   
2       25580.0          21.5  ...  2023.0   충남  강경젓갈축제  1.178086e+09   
3           NaN           NaN  ...  2018.0   강원  강릉커피축제  3.904450e+08   
4       28853.6          39.4  ...  2019.0   강원  강릉커피축제  4.400000e+08   

          사업수익           순원가   시작월   종료월     index_y normalized_index_y  


In [ ]:
df = df.drop(['index_y', 
              'normalized_index_y',
              'normalized_index_x', 
              'df1_original',
              'df2_original',
              'index_x'],
                axis = 1)
df['index'] = df['축제명'] + df['개최년도'].astype(str)
df.to_csv('./data/전국연도별방문자회계정보진짜최종.csv', index=False, encoding='utf-8-sig')
df.columns

# 1차 합성은 실패.
뒤에 합쳐진 년도 2024 등 때문에 기본적인 유사도가 굉장히 올라가있는 상태라 한글 단어가 2~3개만 매칭되더라도 같은 글자라 판단해서 합쳐버림.

ex) 정선 아리랑 2024, 정선 아리랑 2019, 정? 아리랑2024가 있다면 정선 아리랑 2024와 정? 아리랑 2024가 같은 글자라 판단되서 매칭됨.

In [ ]:
from rapidfuzz import process as rf_process
from rapidfuzz import fuzz
import pandas as pd

df1 = pd.read_csv('./전국연도별방문자추이.csv', encoding='euc-kr')
df2 = pd.read_csv('./회계정보전처리.csv', encoding='utf-8')

def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).lower().replace(" ", "")

df1['normalized_index'] = df1['index'].apply(normalize_text)
df2['normalized_index'] = df2['index'].apply(normalize_text)


#개최년도, 회계년도가 같을 경우에만 타겟 하기 위해 추가. 
df1['개최년도'] = df1['개최년도'].astype(int)
df2['회계연도'] = df2['회계연도'].astype(int)

matches = []
for _, row1 in df1.iterrows():
    match = rf_process.extractOne(
        row1['normalized_index'],
        df2['normalized_index'],
        scorer=fuzz.ratio
    )
    if match and match[1] >= 75:                # 일단 75점 이상 끼리만 모아둠.
        matched_row = df2[df2['normalized_index'] == match[0]]
        if not matched_row.empty:
            matched_row = matched_row.iloc[0]  
            if (                                    #'도'와 '지역명'의 앞 2글자가 같아야 함. 
                row1['도'][:2] == matched_row['지역명'][:2] and  
                row1['개최년도'] == matched_row['회계연도']         #회계년도와 개최년도도 같아야 함.
            ):
                matches.append({                    #maches 라는 리스트 안에 dict로 정보를 담아 DF로 만들준비.
                    'df1_index': row1['index'],
                    'df2_index': matched_row['index'],
                    'similarity_score': match[1],
                    '도': row1['도'],
                    '개최년도': row1['개최년도']
                })

matches_df = pd.DataFrame(matches)

print(matches_df.head())
#매치 정보 저장하긴했는데, 의미 있는지는 잘 모르겠지만 하는게 좋다해서 일단 해둠.
matches_df.to_csv('./matched_results_75.csv', index=False, encoding='utf-8-sig')


    df1_index   df2_index  similarity_score   도  개최년도
0  강경젓갈축제2018  강경젓갈축제2018             100.0  충남  2018
1  강경젓갈축제2022  강경젓갈축제2022             100.0  충남  2022
2  강경젓갈축제2023  강경젓갈축제2023             100.0  충남  2023
3  강릉커피축제2018  강릉커피축제2018             100.0  강원  2018
4  강릉커피축제2019  강릉커피축제2019             100.0  강원  2019


In [ ]:
matched_df = pd.read_csv('./matched_results_75.csv')

#df1에 matched_df를 결합해서 df1의 인덱스와 df2의 인덱스 모두 가진 데이터 셋으로 변경.
df2_matched = pd.merge(
    df1, 
    matched_df[['df1_index', 'df2_index']], 
    left_on='index',  
    right_on='df1_index',  
    how='left'  
)

#df1에 들어간 df2의 정보를 바탕으로 df2를 매핑해서 머지함.
final_df = pd.merge(
    df2_matched,  
    df2,  
    left_on='df2_index',  
    right_on='index', 
    how='left' 
)

print(final_df.head())

#완성~ 드랍은 수연님이 해주신 걸로 데이터셋 생성 완료.
final_df.to_csv('./전국연도별방문자회계정보정망정말최종ver.csv', index=False, encoding='utf-8-sig')

      축제명  개최년도  축체기간(일)  (현지인)방문자수  (외지인)방문자수  (외국인)방문자수  (전체)방문자수  일평균 방문자수  \
0  강경젓갈축제  2018        5    50642.0    59291.0      108.0  110041.0  22008.20   
1  강경젓갈축제  2022        5    57971.0    69910.0       19.0  127900.0  25580.00   
2  강경젓갈축제  2023        4    52282.0    71999.0       18.0  124299.0  31074.75   
3  강릉커피축제  2018        5    61368.0    82621.0      279.0  144268.0  28853.60   
4  강릉커피축제  2019        4    98907.0    61246.0      743.0  160896.0  40224.00   

   전년도 일평균 방문자수  일평균 방문자수 증감률  ...    회계연도  지역명   행사축제명           총비용  \
0           NaN           NaN  ...  2018.0   충남  강경젓갈축제  8.250000e+08   
1       22008.2          16.2  ...  2022.0   충남  강경젓갈축제  1.003000e+09   
2       25580.0          21.5  ...  2023.0   충남  강경젓갈축제  1.178086e+09   
3           NaN           NaN  ...  2018.0   강원  강릉커피축제  3.904450e+08   
4       28853.6          39.4  ...  2019.0   강원  강릉커피축제  4.400000e+08   

          사업수익           순원가   시작월   종료월     index_y normalized_index_y  
